# Import Libraries

In [1]:
!!pip install git+https://github.com/huggingface/datasets#egg=datasets --quiet
!pip install -U accelerate --quiet
# !pip install --quiet bitsandbytes-cuda117==0.26.0.post2
# !pip install --quiet transformers[audio,deepspeed,ftfy,onnx,sentencepiece,timm,tokenizers,video,vision]==4.28.1
!pip install bitsandbytes --quiet
!pip install peft --quiet
!pip install trl --quiet

# Wandb Config

In [2]:
import os
import wandb
from fastai.callback.wandb import WandbCallback
from kaggle_secrets import UserSecretsClient

In [1]:
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

# Load Model

In [4]:
from transformers import BitsAndBytesConfig
import torch

# BitsAndBytesConfig to quantize the model int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# base model id to fine-tune
model_id = "Sakonii/distilgpt2-nepali"

# load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)
model.config.pretraining_tp = 1

# load tokenizer, pad short samples with end of sentence token
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="right",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [6]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear4bit(in_features=3072, out_features=768, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [7]:
tokenizer.pad_token

'</s>'

# Import Libraries

In [3]:
from datasets import load_dataset

# download dataset

# for single qsn dataset
dataset = load_dataset("Bibek1129/nepali_SQuAD_single_qsn", split = "train")

# for multiple qsns dataset
dataset = load_dataset("Bibek1129/nepali_SQuAD_multiple_qsns", split = "train")

print(dataset)

# print a sample triplet
print(dataset[0])

# Define Format

In [9]:
# sample['question'] for single qsn dataset

def format_instruction(sample):
    return f"""तपाईं प्रश्नहरू उत्पन्न गर्ने मोडेल हुनुहुन्छ। तपाइँलाई एक सन्दर्भ दिइएको हुन्छ र तपाइँ त्यसमा आधारित प्रश्नहरू उत्पन्न गर्नुहुन्छ।
        ### सन्दर्भ:
        {sample["context"]}
        
        ### प्रश्नहरू:
        {sample["questions"]}
    """

In [10]:
sample = dataset[2445]
print(format_instruction(sample))

तपाईं प्रश्नहरू उत्पन्न गर्ने मोडेल हुनुहुन्छ। तपाइँलाई एक सन्दर्भ दिइएको हुन्छ र तपाइँ त्यसमा आधारित प्रश्नहरू उत्पन्न गर्नुहुन्छ।
        ### सन्दर्भ:
        सन् १९८९ को ३० सेप्टेम्बरमा हजारौं बेलारूसवासीले स्थानीय नेताहरूलाई निन्दा गर्दै मिन्स्कको वरिपरि घुमेर १९८६ को चेर्नोबिल दुर्घटना स्थलको अतिरिक्त सफाइ माग गरे। रेडियोधर्मिता सङ्केत बोकेका बाङ्ग्रा लगाएका १५,००० भन्दा बढी प्रदर्शनकारीहरूले स्थानीय निकायहरूको निषेधाज्ञाको विरूद्ध तुफान वर्षाको माध्यमबाट निषेधित रातो र सेतो बेलारूस राष्ट्रिय झण्डा बोकेका थिए। त्यसपछि तिनीहरूले सरकारको मुख्यालय नजिकको शहरको केन्द्रमा एकत्रित भए, जहाँ वक्ताहरूले गणराज्यको कम्युनिस्ट पार्टीका नेता येफ्रेम सोकोलोभको त्यागपत्र माग गरे र संदूषित क्षेत्रबाट आधा मिलियन मानिसहरूको evakuationको लागि आह्वान गरे।
        
        ### प्रश्नहरू:
        बेलारूसका मानिसहरूले कसरी सफा गर्न चाहन्थे? चेर्नोबिल विपत्ति कहाँ घट्यो? चेर्नोबिल विपत्ति कहिले घट्यो? प्रदर्शनकारीहरूले आफ्नो हातमा कुन प्रतीक लगाए? विरोधको समयमा मौसम कस्तो थियो? 
    


In [11]:
# sample['question'] for single qsn dataset

EOS_TOKEN = tokenizer.eos_token
alpaca_prompt = """तपाईं प्रश्नहरू उत्पन्न गर्ने मोडेल हुनुहुन्छ। तपाइँलाई एक सन्दर्भ दिइएको हुन्छ र तपाइँ त्यसमा आधारित प्रश्नहरू उत्पन्न गर्नुहुन्छ।

### सन्दर्भ:
{}

### प्रश्नहरू:
{}""" + EOS_TOKEN

def formatting_prompts_func(examples):
    context = examples["context"]
    questions = examples["questions"]
    texts = []
    for context,questions in zip(context, questions):
        text = alpaca_prompt.format(context, questions)
        texts.append(text)
    return { "text" : texts, }
pass


In [5]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [13]:
dataset[-100]

{'context': 'जुलाई १९५६ मा नासरले स्विज नहरलाई एकपक्षीय रूपमा राष्ट्रीकृत गरे। चर्चिललाई नयाँ प्रधानमन्त्री नियुक्त गरेको एन्थोनी इडेनको प्रतिक्रिया मिश्रमाथि इस्राएलको आक्रमण गर्ने योजना बनाउन फ्रान्ससँग सहकार्य गर्नु थियो जसले ब्रिटेन र फ्रान्सलाई सैन्य हस्तक्षेप गर्न र नहर फिर्ता लिनको बहाना दिने थियो। इडेनले अमेरिकी राष्ट्रपति डविट डी. इजसेनहाउरलाई उहाँको परामर्शको अभावले क्रोधित पार्यो र इजसेनहाउरले आक्रमणलाई समर्थन गर्न अस्वीकार गरे। इजसेनहाउरको अर्को चिन्ता सोभियत संघसँग विस्तारित युद्धको सम्भावना थियो किनभने यसले मिश्रको पक्षमा हस्तक्षेप गर्ने धमकी दिएको थियो। इजसेनहाउरले अमेरिकी reserves of the British pound बेच्न वाध्य गराएर ब्रिटिश मुद्राको खतम हुनलाई वाध्य गराएर वित्तीय लेभरेज प्रयोग गरे। यद्यपि आक्रमण बल आफ्नो उद्देश्यमा सैन्य सफलता प्राप्त गरे,',
 'questions': 'जब सुइज नहर राष्ट्रीकृत भयो? विन्स्टन चर्चिल पछि ब्रिटिश प्रधानमन्त्री को भयो? इदेनले मिश्रमाथि आक्रमण गर्न कुन देशसँग षडयन्त्र गर्यो? ब्रिटिश पाउन्डको ह्रास हुनलाई कसले पाउन्डको अमेरिकी रिजर्व बिक्री गर्ने निर्णय 

# Initialize PEFT

In [15]:
from peft import LoraConfig

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "c_fc",
        "c_attn",
        "c_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

In [16]:
from peft import prepare_model_for_kbit_training, get_peft_model

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [17]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

# get frozen vs trainable model param statistics
print_trainable_parameters(model)

trainable params: 3992096 || all params: 64671008 || trainable%: 6.17292991629263


# Train

In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'
os.environ['TORCH_USE_CUDA_DSA']='1'

In [4]:
from trl import SFTTrainer
from transformers import TrainingArguments

model_args = TrainingArguments(
    output_dir="nepali-instruct-distil-gpt2",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5000,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    tf32=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

# Supervised Fine-Tuning Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field = "text",
    peft_config=peft_config,
    max_seq_length=1024,
    tokenizer=tokenizer,
    packing=False,
    args=model_args,
)

# train
trainer.train()